# KEGG Drug ID to MeSH Unique ID

In [ ]:
import difflib
import json
from pathlib import Path
import re
import urllib.request
from urllib.error import HTTPError

import pandas as pd
from tqdm import tqdm

## 1. Load KEGG Drug List with Names

### 1.1. Load Data

In [ ]:
kegg_drugs = {}
with open("data/relation/human_KEGG_drug_names.tsv", "r") as f:
    f.readline()  # header
    for line in f.readlines():
        kegg, name, gene, geneid = line.strip().split("\t")
        kegg = kegg.strip('"')
        name = name.strip('"').strip(";")
        # strip text in brackets
        name = re.sub(r"\s\([\s\w/]+\)", "", name)
        kegg_drugs[kegg] = name

### 1.2. Create dataframe

In [ ]:
df = {}
for kegg, name in kegg_drugs.items():
    df[kegg] = {"name": name}

## 2. KEGG to PubChem SID or Chebi ID

### 2.1. Prepare KEGG to Chebi ID

#### 2.1.1. Download & Save

In [ ]:
url = "https://rest.kegg.jp/conv/chebi/drug"
kegg2chebi = {}
with urllib.request.urlopen(url) as response:
    for line in response.readlines():
        line = line.decode()
        kegg, chebi = line.strip().split("\t")
        if kegg in kegg2chebi:  # one KEGG Drug ID can have multiple Chebi IDs
            kegg2chebi[kegg].append(chebi)
        else:
            kegg2chebi[kegg] = [chebi]

In [ ]:
Path("output/kegg2mesh").mkdir(parents=True, exist_ok=True)

In [ ]:
with open("output/kegg2mesh/kegg2chebi.tsv", "w") as f:
    for kegg, chebi in kegg2chebi.items():
        f.write(f"{kegg}\t{','.join(chebi)}\n")

#### 2.1.2. Load

In [ ]:
kegg2chebi = {}
with open("output/kegg2mesh/kegg2chebi.tsv", "r") as f:
    for line in f.readlines():
        kegg, chebi = line.strip().split("\t")
        chebi = str(chebi).split(",")
        kegg2chebi[kegg] = chebi

### 2.2. Prepare KEGG to PubChem SID

#### 2.2.1. Download & Save

In [ ]:
url = "https://rest.kegg.jp/conv/pubchem/drug"
kegg2pubchem = {}
with urllib.request.urlopen(url) as response:
    for line in response.readlines():
        line = line.decode()
        kegg, pubchem = line.strip().split("\t")
        kegg2pubchem[kegg] = pubchem

In [ ]:
with open("output/kegg2mesh/kegg2pubchem.tsv", "w") as f:
    for kegg, pubchem in kegg2pubchem.items():
        f.write(f"{kegg}\t{pubchem}\n")

#### 2.2.2. Load

In [ ]:
kegg2pubchem = {}
with open("output/kegg2mesh/kegg2pubchem.tsv", "r") as f:
    for line in f.readlines():
        kegg, pubchem = line.strip().split("\t")
        kegg2pubchem[kegg] = pubchem

### 2.3. Convert KEGG IDs to PubChem SIDs or Chebi ID

In [ ]:
n_pubchem = 0
n_chebi = 0
n_na = 0
for kegg in kegg_drugs.keys():
    if kegg in kegg2pubchem:
        pubchem = kegg2pubchem[kegg]
    else:
        pubchem = "N/A"

    if kegg in kegg2chebi:
        chebi = kegg2chebi[kegg]
        if len(chebi) == 1:
            chebi = chebi[0]
        else:
            chebi = ",".join(chebi)
    else:
        chebi = "N/A"

    # count
    if pubchem != "N/A":
        n_pubchem += 1
    elif chebi != "N/A":  # will use chebi as fallback
        n_chebi += 1
    else:
        n_na += 1

    # update dataframe
    df[kegg]["chebi"] = chebi
    df[kegg]["pubchem_sid"] = pubchem

print(f"PubChems: {n_pubchem}, Chebis: {n_chebi}, N/As: {n_na}")

## 3. PubChem SID or Chebi ID to PubChem CID

### 3.1. Request CID for SIDs from KEGG

In [ ]:
# PubChem API base url
pug = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"

#### 3.1.1 Download & Save

In [ ]:
url = pug + "/substance/sourceall/KEGG/cids/json"

kegg_sid2cid = {}
with urllib.request.urlopen(url) as response:
    pubchem_chebi_kegg_cids = json.load(response)
for item in pubchem_chebi_kegg_cids["InformationList"]["Information"]:
    if "CID" in item:
        assert len(item["CID"]) == 1
        kegg_sid2cid[item["SID"]] = item["CID"][0]

In [ ]:
with open("output/kegg2mesh/kegg_sid2cid.tsv", "w") as f:
    f.writelines([f"{str(k)}\t{str(v)}\n" for k, v in kegg_sid2cid.items()])

#### 3.1.2. Load

In [ ]:
kegg_sid2cid = {}
with open("output/kegg2mesh/kegg_sid2cid.tsv", "r") as f:
    for line in f.readlines():
        kegg_sid, cid = line.strip().split("\t")
        kegg_sid2cid[int(kegg_sid)] = int(cid)

### 3.2. Convert KEGG SID or Chebi ID to PubChem CIDs

In [ ]:
n_cid = 0
for kegg, value in df.items():
    pubchem_sid = value["pubchem_sid"]
    chebi_id = value["chebi"]
    cid = "N/A"

    if pubchem_sid != "N/A":  # from kegg sid
        pubchem_sid = int(pubchem_sid[len("pubchem:"):])
        if pubchem_sid in kegg_sid2cid:
            cid = kegg_sid2cid[pubchem_sid]
            n_cid += 1

    elif chebi_id != "N/A":  # from chebi id; use pubchem api
        url = pug + f"/compound/xref/RegistryID/{chebi_id}/cids/json"
        try:
            with urllib.request.urlopen(url) as response:
                pubchem_chebi_cids = json.load(response)

            id_list = pubchem_chebi_cids["IdentifierList"]
            if "CID" in id_list:
                assert len(id_list["CID"]) == 1
                cid = id_list["CID"][0]
                n_cid += 1

        except HTTPError:  # excepting one key not found in pubchem
            pass

    value["pubchem_cid"] = cid

print(f"Converted {n_cid} out of {len(df)}")

## 4. PubChem CIDs to MeSH Unique IDs

### 4.1. Convert CIDs to MeSH Heading and Terms

#### 4.1.1. Download & Save

In [ ]:
cid2kegg = {}
for kegg, v in df.items():
    cid = str(v["pubchem_cid"])
    if cid != "N/A":
        cid2kegg[cid] = kegg

In [ ]:
n_batch = 200

kegg2mesh = {}
cid_list = list(cid2kegg.keys())
for i in tqdm(range(0, len(cid_list), n_batch)):
    cid_batch = cid_list[i:i+n_batch]
    esummary = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pccompound&id={','.join(cid_batch)}&retmode=json"
    with urllib.request.urlopen(esummary) as response:
        results = json.load(response)
    # parse results
    result = results["result"]
    uids = result["uids"]
    for uid in uids:
        cid = str(result[uid]["cid"])
        mesh_headings = result[uid]["meshheadinglist"]
        mesh_terms = result[uid]["meshtermlist"]
        kegg2mesh[cid2kegg[cid]] = {
            "mesh_headings": mesh_headings,
            "mesh_terms": mesh_terms
        }

In [ ]:
# save
with open("output/kegg2mesh/kegg2mesh.tsv", "w") as f:
    for kegg, value in kegg2mesh.items():
        headings = ','.join(['"'+heading+'"' for heading in value['mesh_headings']])
        terms = ','.join(['"'+term+'"' for term in value['mesh_terms']])
        if headings and terms:
            f.write(f"{kegg}\t{headings}\t{terms}\n")

#### 4.1.2. Load

In [ ]:
kegg2mesh = {}
with open("output/kegg2mesh/kegg2mesh.tsv", "r") as f:
    for line in f.readlines():
        kegg, headings, terms = line.strip().split("\t")
        headings = eval("[" + headings + "]")
        terms = eval("[" + terms + "]")
        kegg2mesh[kegg] = {
            "mesh_headings": headings,
            "mesh_terms": terms,
        }

#### 4.1.3. Convert CIDs to MeSH headings & terms

In [ ]:
for kegg, value in df.items():
    mesh_headings = []
    mesh_terms = []
    if kegg in kegg2mesh:
        mesh_headings = kegg2mesh[kegg]["mesh_headings"]
        mesh_terms = kegg2mesh[kegg]["mesh_terms"]
    value.update({
        "mesh_headings": mesh_headings,
        "mesh_terms": mesh_terms
    })

###  4.2. Fetch Labels for MeSH Unique IDs

#### 4.2.1. Load BioConceptVec Vocab

In [ ]:
with open("data/embeddings/vocab_chem.csv", "r") as f:
    f.readline()
    chems = [line[len("Chemical_MESH_"):].strip() for line in f.readlines()]

#### 4.2.2. Fetch labels for MeSH unique ids from MeSH RDF API

In [ ]:
sparql = "https://id.nlm.nih.gov/mesh/sparql?query="
headers = {"accept": "application/sparql-results+json"}
n_batch = 400

out_dict = {}
for i in tqdm(range(0, len(chems), n_batch)):
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX mesh: <http://id.nlm.nih.gov/mesh/>

    SELECT *
    FROM <http://id.nlm.nih.gov/mesh>
    WHERE {{
        VALUES ?meshId {{ {' '.join(['mesh:'+ chem for chem in chems[i:i+n_batch]])} }}
        ?meshId rdfs:label ?meshLabel .
    }}
    """
    url = sparql + urllib.parse.quote(query)
    request = urllib.request.Request(url, headers=headers)

    with urllib.request.urlopen(request) as response:
        label = json.load(response)

    for result in label["results"]["bindings"]:
        chem_id = result["meshId"]["value"][len("http://id.nlm.nih.gov/mesh/"):]
        value = result["meshLabel"]["value"]
        out_dict[chem_id] = value

#### 4.2.3. Save

In [ ]:
with open("output/kegg2mesh/mesh_labels.tsv", "w") as f:
    for key, value in out_dict.items():
        value = value.replace("\t", " ")
        f.write(f"Chemical_MESH_{key}\t{value}\n")

#### 4.2.4. Load

In [ ]:
# load mesh concept values
mesh_label2id = {}
concepts = []
with open("output/kegg2mesh/mesh_labels.tsv", "r") as f:
    for line in f.readlines():
        mesh_id, concept = line.strip().split("\t")
        mesh_id = mesh_id[len("Chemical_MESH_"):]
        mesh_label2id[concept] = mesh_id
        concepts.append(concept)

### 4.3. Convert

#### 4.3.1. Convert

In [ ]:
cutoff = 0.85  # may need some tweaking

for kegg, value in tqdm(df.items()):
    headings = value["mesh_headings"]
    # terms = value["mesh_terms"]
    name = value["name"]
    mesh_id = None
    score = 0

    conv_method = "exact_"
    for heading in headings:
        if heading in mesh_label2id:
            mesh_id = mesh_label2id[heading]
            label = heading
            conv_method += "heading"
            break
    else:
        if name in mesh_label2id:
            mesh_id = mesh_label2id[name]
            label = name
            conv_method += "name"

    if not mesh_id:  # no exact match
        conv_method = "fuzzy_"
        for heading in headings:
            if match := difflib.get_close_matches(heading, concepts, n=1, cutoff=cutoff):
                label = match[0]
                mesh_id = mesh_label2id[label]
                conv_method += "heading"
                score = difflib.SequenceMatcher(None, heading, label).ratio()
                break
        else:
            if match := difflib.get_close_matches(name, concepts, n=1, cutoff=cutoff):
                label = match[0]
                mesh_id = mesh_label2id[label]
                conv_method += "name"
                score = difflib.SequenceMatcher(None, name, label).ratio()
            else:  # failed
                label = "N/A"
                mesh_id = "N/A"
                conv_method = "failed"

    value.update({
        "mesh_id": mesh_id,
        "conv_method": conv_method,
        "mesh_label": label,
        "score": score,
    })

#### 4.3.2. Save as dataframe

In [ ]:
dataframe = pd.DataFrame(list(df.values()))
dataframe.insert(0, "kegg_id", df.keys())

In [ ]:
dataframe.to_csv("output/kegg2mesh/kegg2mesh_df.csv", index=None)